## Exemplo 02. O problema de transporte

### Custos de transporte por tonelada, produção e Demanda

| SID/FAB|Farmington [$-Ton]|Detroit [$-Ton]|Lansing [$-Ton]|Winsor [$-Ton]| St. Louis [$-Ton] | Fremont [$-Ton]| Lafayette [$-Ton]| **Produção [Ton]**|
|:----:|:------:|:------:|:------:|:------:|:------:|:------:|:------:| :------:|
|Gary| 39 | 14 | 11 | 14 | 16 | 82 | 8 | 1400 |
|Cleveland| 27 | 9 | 12 | 9 | 26 | 95 | 17 | 2600 |
|Pittsburg| 24 | 14 | 17 | 13 | 28 | 99 | 20 | 2900 |
|**Demanda [Ton]**| 900 | 1200 | 600 | 400 | 1700 | 1100 | 1100 | $ \sum{6900}$ |

![Transporte](../Figuras/1.2.png)

## Formulação General

### Conjuntos
- $ S: \text{Conjunto de siderúrgicas.}$
- $ F: \text{Conjunto de fábricas.}$

### Parâmetros
- $ c_{s,f} : \text{Custo de tranporte por tonelada da siderúrgica s para a fábrica f.} $
- $ p_{s} : \text{Produção da siderúrgica en toneladas.} $
- $ d_{f} : \text{Demanda da fábrica f em toneladas.} $

### Variáveis de decisão

- $x_{s,f} : \text{Toneladas transportadas da siderúrgica s para a fábrica f.}$ 

### Função objetivo 

- $ \mathrm{Min} \sum\limits_{s \in S} \sum\limits_{f \in F} c_{s,f} \text{ } x _{s,f}$

### Restrições 

- $ \sum\limits_{f \in F} x_{s,f} = p_{s}  \;\;\;\;\;\; ;\forall s \in S $
- $ \sum\limits_{s \in S} x_{s,f} = d_{f} \;\;\;\;\;\; ;\forall f \in F $
- $ x_{s,f} \geq 0 \;\; ;\forall s \in S \text{, } f \in F $



## Preparação dos dados de entrada

In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('../Datos/1.2.xlsx','Datos', header=0, index_col=0)
print(df)

           Farmington  Detroit  Lansing  Winsor  St. Louis  Fremont  Lafayette
Gary               39       14       11      14         16       82          8
Cleveland          27        9       12       9         26       95         17
Pittsburg          24       14       17      13         28       99         20


In [4]:
df_2 = pd.read_excel('../Datos/1.2.xlsx','Prod', header=0, index_col=0)
print(df_2)

           Producao
Gary           1400
Cleveland      2600
Pittsburg      2900


In [5]:
df_3 = pd.read_excel('../Datos/1.2.xlsx','Dem', header=0, index_col=0)
print(df_3)

            Demanda
Farmington      900
Detroit        1200
Lansing         600
Winsor          400
St. Louis      1700
Fremont        1100
Lafayette      1000


### Listas e dictionarios

In [6]:
SID = list(df.index.map(str))
FAB = list(df.columns.map(str))
print(SID, FAB)

['Gary', 'Cleveland', 'Pittsburg'] ['Farmington', 'Detroit', 'Lansing', 'Winsor', 'St. Louis', 'Fremont', 'Lafayette']


In [7]:
costos = {(s,f):df.at[s,f] for s in SID for f in FAB}

In [8]:
producao = df_2.to_dict()['Producao']

In [9]:
demanda = df_3.to_dict()['Demanda']

In [10]:
print(SID)
print(FAB)

['Gary', 'Cleveland', 'Pittsburg']
['Farmington', 'Detroit', 'Lansing', 'Winsor', 'St. Louis', 'Fremont', 'Lafayette']


In [11]:
print(producao)
print(demanda)
print(costos)

{'Gary': 1400, 'Cleveland': 2600, 'Pittsburg': 2900}
{'Farmington': 900, 'Detroit': 1200, 'Lansing': 600, 'Winsor': 400, 'St. Louis': 1700, 'Fremont': 1100, 'Lafayette': 1000}
{('Gary', 'Farmington'): 39, ('Gary', 'Detroit'): 14, ('Gary', 'Lansing'): 11, ('Gary', 'Winsor'): 14, ('Gary', 'St. Louis'): 16, ('Gary', 'Fremont'): 82, ('Gary', 'Lafayette'): 8, ('Cleveland', 'Farmington'): 27, ('Cleveland', 'Detroit'): 9, ('Cleveland', 'Lansing'): 12, ('Cleveland', 'Winsor'): 9, ('Cleveland', 'St. Louis'): 26, ('Cleveland', 'Fremont'): 95, ('Cleveland', 'Lafayette'): 17, ('Pittsburg', 'Farmington'): 24, ('Pittsburg', 'Detroit'): 14, ('Pittsburg', 'Lansing'): 17, ('Pittsburg', 'Winsor'): 13, ('Pittsburg', 'St. Louis'): 28, ('Pittsburg', 'Fremont'): 99, ('Pittsburg', 'Lafayette'): 20}


## Modelo computacional no Pyomo

In [12]:
from pyomo.environ import *
model = ConcreteModel(name='Transporte')

### Conjuntos

- $ S: \text{Conjunto de siderúrgicas.}$
- $ F: \text{Conjunto de fábricas.}$

In [24]:
S = (SID)
F = (FAB)
print(S)

['Gary', 'Cleveland', 'Pittsburg']


### Parâmetros

- $ c_{s,f} : \text{Custo de tranporte por tonelada da siderúrgica s para a fábrica f.} $
- $ p_{s} : \text{Produção da siderúrgica en toneladas.} $
- $ d_{f} : \text{Demanda da fábrica f em toneladas.} $

In [23]:
c = (costos)
p = (producao)
d = (demanda)
print(c)

{('Gary', 'Farmington'): 39, ('Gary', 'Detroit'): 14, ('Gary', 'Lansing'): 11, ('Gary', 'Winsor'): 14, ('Gary', 'St. Louis'): 16, ('Gary', 'Fremont'): 82, ('Gary', 'Lafayette'): 8, ('Cleveland', 'Farmington'): 27, ('Cleveland', 'Detroit'): 9, ('Cleveland', 'Lansing'): 12, ('Cleveland', 'Winsor'): 9, ('Cleveland', 'St. Louis'): 26, ('Cleveland', 'Fremont'): 95, ('Cleveland', 'Lafayette'): 17, ('Pittsburg', 'Farmington'): 24, ('Pittsburg', 'Detroit'): 14, ('Pittsburg', 'Lansing'): 17, ('Pittsburg', 'Winsor'): 13, ('Pittsburg', 'St. Louis'): 28, ('Pittsburg', 'Fremont'): 99, ('Pittsburg', 'Lafayette'): 20}


### Variáveis de decisão

- $x_{s,f} : \text{Toneladas transportadas da siderúrgica s para a fábrica f.}$ 
- $ x_{s,f} \geq 0 \;\; ;\forall s \in S \text{, } f \in F $

In [15]:
model.x = Var(S,F, domain = NonNegativeReals)

### Função objetivo

- $ \mathrm{Min} \sum\limits_{s \in S} \sum\limits_{f \in F} c_{s,f} \text{ } x _{s,f}$



In [16]:
def obj_rule(m):
    return sum(c[s,f]*m.x[s,f] for s in S for f in F)
model.obj = Objective(rule = obj_rule, sense = minimize)

### Restrições

- $ \sum\limits_{f \in F} x_{s,f} = p_{s}  \;\;\;\;\;\; ;\forall s \in S $
- $ \sum\limits_{s \in S} x_{s,f} = d_{f} \;\;\;\;\;\; ;\forall f \in F $

In [17]:
def pro_cost_rule(m,s):
    return sum(m.x[s,f] for f in F) == p[s]
model.restricao_1 = Constraint(S, rule=pro_cost_rule)

In [18]:
def dem_cost_rule(m,f):
    return sum(m.x[s,f] for s in S) == d[f]
model.restricao_2 = Constraint(F, rule=dem_cost_rule)

In [19]:
model.pprint()

5 Set Declarations
    restricao_1_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Gary', 'Cleveland', 'Pittsburg'}
    restricao_2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {'Farmington', 'Detroit', 'Lansing', 'Winsor', 'St. Louis', 'Fremont', 'Lafayette'}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   21 : {('Gary', 'Farmington'), ('Gary', 'Detroit'), ('Gary', 'Lansing'), ('Gary', 'Winsor'), ('Gary', 'St. Louis'), ('Gary', 'Fremont'), ('Gary', 'Lafayette'), ('Cleveland', 'Farmington'), ('Cleveland', 'Detroit'), ('Cleveland', 'Lansing'), ('Cleveland', 'Winsor'), ('Cleveland', 'St. Louis'), ('Cleveland', 'Fremont'), ('Cleveland', 'Lafayette'), ('Pittsburg', 'Farmington'), ('Pittsburg', 'Detroit'), ('Pittsbur

### Solução

In [20]:
Resultado = SolverFactory('cbc', executable='C:/Solvers/cbc.exe').solve(model)

In [21]:
model.display()

Model Transporte

  Variables:
    x : Size=21, Index=x_index
        Key                         : Lower : Value  : Upper : Fixed : Stale : Domain
           ('Cleveland', 'Detroit') :     0 : 1200.0 :  None : False : False : NonNegativeReals
        ('Cleveland', 'Farmington') :     0 :    0.0 :  None : False : False : NonNegativeReals
           ('Cleveland', 'Fremont') :     0 :    0.0 :  None : False : False : NonNegativeReals
         ('Cleveland', 'Lafayette') :     0 :  400.0 :  None : False : False : NonNegativeReals
           ('Cleveland', 'Lansing') :     0 :  600.0 :  None : False : False : NonNegativeReals
         ('Cleveland', 'St. Louis') :     0 :    0.0 :  None : False : False : NonNegativeReals
            ('Cleveland', 'Winsor') :     0 :  400.0 :  None : False : False : NonNegativeReals
                ('Gary', 'Detroit') :     0 :    0.0 :  None : False : False : NonNegativeReals
             ('Gary', 'Farmington') :     0 :    0.0 :  None : False : False : NonNe

In [22]:
for s in S:
    for f in F:
        quantidade = model.x[s,f]()
        if quantidade > 0:
            print(f'De {s}, para {f} vai transportar {quantidade} toneladas')

De Gary, para Fremont vai transportar 1100.0 toneladas
De Gary, para Lafayette vai transportar 300.0 toneladas
De Cleveland, para Detroit vai transportar 1200.0 toneladas
De Cleveland, para Lansing vai transportar 600.0 toneladas
De Cleveland, para Winsor vai transportar 400.0 toneladas
De Cleveland, para Lafayette vai transportar 400.0 toneladas
De Pittsburg, para Farmington vai transportar 900.0 toneladas
De Pittsburg, para St. Louis vai transportar 1700.0 toneladas
De Pittsburg, para Lafayette vai transportar 300.0 toneladas
